Zadanie 1.

Zaprojektuj klasę dla $kd$-drzewa.

In [40]:
import math
# przy wykonaniu tego zadania korzystam z biblioteki pandas (również, ponieważ trochę ją poznałem),
# ponieważ dostarcza pewne ułatwiające funkcjonalności, jak:
# DataFrames - bardzo wygodny pojemnik na dane, funkcja '.iloc' i '.sort_values'.

import pandas as pd
from binarytree import tree, Node

class kd_Tree:

    def __init__(self, data):
        self.data = data
        self.tree = None

    def _build(self, points, depth):

        # wyliczamy wymiar k (zakładamy, że dane otrzymujemy jako dataframe) - pandas
        k = len(points.columns)

        # ustalamy wybór kolumny (ma będą wybierane po kolei z indeksami [0], [1], [0] , [1] itd.)
        _axis = depth % k

        # wybieramy kolumnę
        _column = points.columns[_axis]
        if len(points) == 0:
            return None

        # sortyjemy DataFrame rosnąco, aby obliczyć medianę
        objects_list = points.sort_values(by= [_column], ascending= True)

        # obliczamy medianę
        median_idx = int((len(objects_list)//2))

        node = Node(int(round(objects_list.iloc[median_idx][_column],3)))
        node.left = self._build(objects_list.iloc[0:median_idx], depth+1)
        node.right = self._build(objects_list.iloc[median_idx+1:], depth+1)
        return node

    def build(self):
        self.tree = self._build(self.data, depth= 0)

test_df = pd.DataFrame(data= [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2],[6,1],[10,2],[11,10],[12,5]], columns= ['X', 'Y'])

kd = kd_Tree(test_df)
kd.build()
print(kd.tree)

# Wybrałem sposób tworzenia drzewa przy użyciu Node (binarytree), więc w węzłach muszą znajdywać się pojedyńcze liczby,
# Zbiór danych, jednak pozwala jednoznacznie stwierdzić jak wygląda podział, drzewo można by było równoważnie zapisać z
# tuple'ami w węzłach, lecz sposób podany przeze mnie, jest również dopuszczony. Trzeba pamiętać, że (od góry) wartości
# występują w kolejności: wartość z kolumny X, Y, X, Y ... Więc na samej górze jest [8,1] , później [2,3] i [9,6] itd.


      ____8_____
     /          \
    3__         _6
   /   \       /  \
  7     5     12   11
 /     /     /
1     7     2



Zadanie 2.

Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania _http_ (symulacja),
w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone:
- czy użytkownik może wysłać danego typu żądania
- czy żądanie nie dotyczy pliku
- czy liczba żądań na minutę nie jest przekroczona
- czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika
- czy przesłany formularz nie jest próbą `sql incjection`

In [22]:
#Regex for detection SQL: "\'|\"|\(|\[|\*|=| |\\|/|;|-"i

from datetime import datetime
import time

class httpSimRequest:
    def __init__(self, Type: str, File: str, Role: str, IsLogged: bool):
        self.httpReq = {
            'Type': Type,
            'File': File,
            'Minute': datetime.now().strftime('%M'),
            'Role': Role,
            'IsLogged': IsLogged,
            'Parameters': {
                'Username': 'John123',
                'Password': "; select true; --"  # Przykład SQL injection - nasze rozwiązanie powinno odrzucić takie dane
            }
             }

request = httpSimRequest('GET', 'FILE', 'ADMIN', True)
print(request.httpReq['Parameters']['Password'])

; select true; --


In [7]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional
# from datetime import datetime
#
# class httpSimRequest:
#     def __init__(self, Type, File):
#         self.httpReq = {
#             'Type': Type,
#             'File': File,
#             'Date': datetime.now().strftime("%d/%m/%Y %H:%M:%S")
#
#         }


class Handler(ABC):

    @abstractmethod
    def set_next(self, handler: Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass


class AbstractHandler(Handler):

    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return None

class TypeHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Banana":
            return f"Monkey: I'll eat the {request}"
        else:
            return super().handle(request)


class FileHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Nut":
            return f"Squirrel: I'll eat the {request}"
        else:
            return super().handle(request)


class RequestsPerMinuteHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "MeatBall":
            return f"Dog: I'll eat the {request}"
        else:
            return super().handle(request)

class SqlInjectionHandler(AbstractHandler):
    def handle(self, request: Any) -> str:


def client_code(handler: Handler) -> None:

    for food in ["Nut", "Banana", "Cup of coffee"]:
        print(f"\nClient: Who wants a {food}?")
        result = handler.handle(food)
        if result:
            print(f"  {result}", end="")
        else:
            print(f"  {food} was left untouched.", end="")

monkey = MonkeyHandler()
squirrel = SquirrelHandler()
dog = DogHandler()

monkey.set_next(squirrel).set_next(dog)

# The client should be able to send a request to any handler, not just the
# first one in the chain.
print("Chain: Monkey > Squirrel > Dog")
client_code(monkey)
print("\n")

print("Subchain: Squirrel > Dog")
client_code(squirrel)

Chain: Monkey > Squirrel > Dog

Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Monkey: I'll eat the Banana
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.

Subchain: Squirrel > Dog

Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Banana was left untouched.
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.

Zadanie 3.

Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
* a + b = c (poprawne),
* (a + b = c (niepoprawne),
* a + = c (niepoprawne).